# 2.4 通过优化RAG过程提升知识库问答准确度
## 🚄  前言

在2.3章节中，你已经可以通过自动化评测发现答疑机器人的问题，也可以初步定位到可能是哪个环节上存在问题。

针对这些问题，你也可以使用2.2章节中介绍的提示词优化技巧，在一定程度上改善机器人的回答质量。

然而，在实际应用中，有时即使进行了细致的提示词优化，也可能遇到机器人无法准确引用知识库中已有信息的情况。这通常意味着你需要进一步探究RAG流程中的其他潜在改进点。


## 🍁 课程目标
学完本课程后，你将能够：

*   了解 RAG 的实现原理与技术细节
    
*   了解 RAG 应用常见的问题与处理方式建议
    
*   通过案例，动手改进RAG应用效果
    

<style>
    table {
      width: 80%;
      margin: 20px auto; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>

## 🐞 1. 棘手的线上问题

### 1.1. 为什么知识库不起作用
  
在使用知识库增强了新人答疑机器人之后，我们仍然有可能遇到一些现象，即知识库里已经有了知识，但是大模型回答仍然不准确。

例如下面的现象：

|   **现象**    |   **问题**    |   **预期**    |
|   --- |   --- | ---   |
|**现象 1： 大模型无法理解文档里的图片信息**|对于文档中的图片，无法理解图片中的信息<br><img src="https://gw.alicdn.com/imgextra/i3/O1CN015tZv6M1wKsFUK7we2_!!6000000006290-0-tps-2554-1252.jpg" alt="工作流编排" width="500px">|实际上是：需要返回图片内的信息<br><img src="https://gw.alicdn.com/imgextra/i1/O1CN01MdZiav1FFoORwSkDN_!!6000000000458-0-tps-1860-900.jpg" alt="工作流编排" width="500px">|
|**现象 2： 大模型无法理解一段文本内容的详细语义**|对于多文档、复杂目录结构文档，一段“注意事项”文本大模型无法理解到具体是什么的注意事项：<br><img src="https://gw.alicdn.com/imgextra/i2/O1CN01HTZkar1xXzCh9PpWN_!!6000000006454-0-tps-2620-1134.jpg" alt="工作流编排" width="600px">|实际上是：内容开发工程师在工作流程中评审环节的注意事项<br><img src="https://gw.alicdn.com/imgextra/i3/O1CN01IXMxiM246EVxvzbcQ_!!6000000007341-0-tps-2374-1104.jpg" alt="工作流编排" width="600px">|
|**现象 3： 大模型无法理解表格单元格的详细语义**|对于复杂多行、跨页的表格，单元格内的文本与表头丢失语义关联，一段单元格文本大模型无法理解到具体是什么条目：<br><img src="https://gw.alicdn.com/imgextra/i2/O1CN012w65dh1eqKMipUPqN_!!6000000003922-0-tps-2608-1142.jpg" alt="工作流编排" width="600px">|实际上是：先根据表头找到部门为 IT，然后根据表头找到人员是张伟，然后再根据表头找到工作职责<br><img src="https://gw.alicdn.com/imgextra/i1/O1CN018M8a4W1KijTzNpv3w_!!6000000001198-0-tps-1882-1064.jpg" alt="工作流编排" width="600px">|

此时你可以尝试通过优化 RAG 过程，提升知识库内容检索准确度。


### 1.2. RAG 工作原理浅析

你可参考下图，回顾我们学过的 RAG 的工作过程。

图：大模型RAG基本工作流
<div align="center">
<img src="https://gw.alicdn.com/imgextra/i3/O1CN01PKaHWH23GTl7IZDSu_!!6000000007228-0-tps-3018-1420.jpg" alt="工作流编排" width="800px">
</div>

如图所示，RAG主要由两个部分构成：

### 1.2.1. 建立索引

将原始的文档语料，通过解析、切片、向量化处理等方式，转化为大模型易于理解与处理的向量文本，然后存储在向量数据库。几个关键的过程及目的如下：
    
*   **Parse 语料**：也被称为文档解析。

    * **处理过程**：使用文档解析工具，将原始多种格式的知识库文档（如PDF、Word等），统一解析为纯文本数据，清洗和提取出有用的信息。
    * **为什么需要文档解析**：这一步骤确保文本数据的一致性和有效性，去除不必要的格式和噪音，使得后续处理更为高效，降低后续处理难度。


*   **Chunks 句子/段落**：也被称为文档切片或文档分割。

    * **处理过程**：使用文本切片工具，将长文本进行数据处理，分割成更短、更易于处理的片段（chunk）。
    * **为什么需要文档切片**：大模型处理的输入数据有Token长度的限制，过长的数据大模型无法处理，同时处理更长的文本序列需要更多的计算资源。通过将长文档分割成更小的块，可以更高效地进行计算，也可以提高检索和生成的效率，确保模型能够更好地理解和利用内容。
    * **文档切片方法**：常见的文档切片方法有按字符分割、按token长度分割、按语义分割等，这几种方式的处理复杂度与资源要求逐渐提升，但是模型处理效果也逐渐提升。若仅按照单一字符或token长度进行文本简单分割，很容易使文本的语义信息丧失，这样在回答问题时可能会出现偏差，因此，为了确保语义的准确性，我们应该尽量将文本分割为包含完整语义的段落或单元。

*   **Embeddings 嵌入**：也被称为文本向量化。文本向量化是将文本数据转换成数字形式的过程。
    * **处理过程**：使用embedding文本嵌入模型，将自然语言文本片段chunk转换为向量表示，例如“猫”转化为[0.2, 0.8, 0.5]。
    * **为什么需要向量化**：因为计算机擅长处理数字而不是文字，通过向量化，我们可以让计算机理解和处理自然语言。嵌入过程通过 embedding 模型，将自然语言转换为高维空间中的向量数据，保留文本的语义信息，这种语义理解对自然语言处理任务非常重要，使得后续的相似度计算和信息检索变得更加有效。例如向量化后，"猫" -> [0.2, 0.8, 0.5]，"狗" -> [0.3, 0.9, 0.4]，"苹果" -> [0.9, 0.1, 0.2]，猫和狗在向量空间距离上更近，计算机从而能够判断出语义更相关。
        
*   **Index 向量数据库**：也被称为构建向量索引。

    * **处理过程**：将经过embedding嵌入的向量和原始文本片段以键值对的形式，存储在专用向量数据库中，并为存储的向量构建索引。

    * **为什么需要使用向量数据库构建索引**：向量数据库提供了存储、管理和检索大规模向量数据的能力。向量数据库和索引结构极大地优化了相似性搜索的效率，通过索引系统可以快速找到与查询向量最相似的向量，而不需要逐个对比整个数据库中的庞大向量数据，这样使得我们可以在大量数据中高效找到与用户查询相关的片段，从而提升响应速度和准确性。


### 1.2.2. 检索生成

大模型结合用户输入的问题与向量数据库中的召回结果给出回答。几个关键的过程如下：
*   **向量检索**：系统获取用户输入，将用户提出的自然语言文本问题转化为向量表示，计算出用户问题与向量数据库中的文档块之间的相似度。选择相似度最高的K个文档块（K值可以自己设置）召回，作为回答当前问题的知识。通过这种方式，可以最大限度地减少生成模型的输入信息噪声，提高回答的准确性和质量。
        
*   **生成最终回复**：系统将用户的问题和检索到的K个相关文档块知识，合并到预设的提示词模板中提交给大模型，大模型生成最终的回复。
        

### 1.3. 检索不准确的可能原因

回到本课程的示例项目，部分 RAG 环节用到的工具或技术：

*   Parse 语料：使用的 LlamaIndex 提供的[SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/api_reference/readers/simple_directory_reader/)，根据文件扩展名自动选择对应的格式，使用简单、解析快。但是仅是做简单文本读取，文档原有结构（如目录顺序/层级/表格等）无法保留。
    
*   Chunks 句子/段落：在调用`VectorStoreIndex.from_documents`构建索引过程中，对文本进行分割。但是分割的较简单，分割过程中会丢失文档语义信息。
    

结合上述 RAG 工作过程和我们的示例项目 RAG 架构，回到我们具体的问题，可能的原因分析如下：
|  **问题现象**  |  **原因**  |  **对应 RAG 的问题环节**  |
| --- | --- | --- |
|  **现象 1： 大模型无法理解文档里的图片信息** <br>对于文档中的图片，无法理解图片中的信息  |  **检索只召回了文档里的文本信息，未召回文档里的图片里的信息** <br>以咨询上线异常告警信息为例，信息隐藏在文档的图片内，解析时实际上没有解析出图片的内容，所以召回的信息也没有图片的信息，因此大模型无法答复出准确的告警信息： <br><img src="https://gw.alicdn.com/imgextra/i3/O1CN015tZv6M1wKsFUK7we2_!!6000000006290-0-tps-2554-1252.jpg" alt="工作流编排" width="600px"> <br>**具体到技术实现细节，原因是：** 文档解析时，仅按照简单的文本分割方式，仅仅解析了文档中的文本，对于图片以及图片里的信息，没有正确解析到  |  文档解析（Parse 语料）  |
|  **现象 2： 大模型无法理解一段文本内容的详细语义** <br>对于公司不同岗位的指导书文档，文档里的重名标题的内容，无法准确识别理解  |  **检索虽然已经召回了切片内容但是缺少更多语义信息** <br>以咨询内容开发工程师评审的注意事项为例，系统虽然召回了注意事项，但是无法识别这个注意事项是什么的注意事项，因此无法生成准确的回答： <br><img src="https://gw.alicdn.com/imgextra/i2/O1CN01HTZkar1xXzCh9PpWN_!!6000000006454-0-tps-2620-1134.jpg" alt="工作流编排" width="600px"> <br>**具体到技术实现细节，原因是：** 文本切片时，仅按照简单的文本分割方式，截取了段落的内容，缺少上下文的信息指引，系统不知道这里切片的注意事项，准确来说是“内容开发工程师评审时”的注意事项  |  文档切片（chunks）  |
|  **现象 3： 大模型无法理解表格单元格的详细语义** <br>对于公司信息表，单元格无法被正确识别和区分  |  **检索虽然已经召回了切片内容但是缺少更多语义信息** <br>以咨询 it 张伟工作职责为例，系统虽然召回了张伟的工作职责，但是公司不同部门有多个同名的张伟，召回的信息大模型无法区分：<br> <img src="https://gw.alicdn.com/imgextra/i2/O1CN012w65dh1eqKMipUPqN_!!6000000003922-0-tps-2608-1142.jpg" alt="工作流编排" width="600px"> <br>**具体到技术实现细节，原因是：** 文本切片时，仅按照简单的文本分割方式，逐行按顺序切片了单元格的内容，没有切片文本行列和表头的上下文信息，缺少上下文的信息指引，系统不知道这里切片的张伟工作职责，准确来说是“IT部张伟”的工作职责  |  文档切片（chunks）  |


<style>
    table {
      width: 80%;
      margin: 20px auto; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>

## 💡 2. 优化知识库问答能力的方法
结合前面的分析过程，从用户提问和最终大模型生成答案的角度来看，我们 RAG 应用的检索召回的知识，应从用户提问的习惯角度去出发，并且要尽量符合大模型的解析方式。同时，我们对用户的可能存在问题的使用场景、可能的提问方式、可能想要的答案要有初步的预判，方可更好的调优我们的 RAG 应用。

### 2.1. 优化文档解析方法

**实现方法**：在我们的项目代码中，我们可以将系统默认的文档解析方式`SimpleDirectoryReader`修改为我们自定义的工具，通过调用我们自定义的解析工具将数据传入，从而实现解析方式改造。

**文档解析优化整体思路**：

Markdown格式因其简洁性和易于解析的特点，被广泛认为是LLM（大型语言模型）友好的文档格式。Markdown通过明确的标记语法，帮助模型更好地理解文档结构和内容，从而提高信息提取的准确性和效率。

我们可以优化文档解析方式，将原有文件解析为统一的 markdown 格式，尽可能保留文档原有完整结构（如目录顺序/层级/表格/图片信息等）。对于文档中的图片，在解析时使用图片理解模型或工具，提取图片中的信息，并补充到解析后的 markdown 文本中。

**具体思路**：

*   **目录顺序层级**：解析后的文本，使用 markdown 形式尽量保留完整顺序与层级，例如：

<table  width="80%" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>
内容开发工程师 岗位类型 大类: 技术大类 细分类型: 综合技术岗位 工作职责 核心职责 结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。 详细职责 1. 内容研究与分析 对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。 2. 教材和课程开发 根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。

</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
# 内容开发工程师
## 岗位类型
- 大类: 技术大类
- 细分类型: 综合技术岗位
## 工作职责
### 核心职责
结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。
### 详细职责
#### 1. 内容研究与分析
对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。
#### 2. 教材和课程开发
根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。
```
</td>
</tr>
</tbody>
</table>


*   **表格**：解析后的文本，使用 markdown 形式尽量保留表格可读性，例如：
<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>
部门 员工姓名 员工主管 工位 工号 岗位 职位 电话 邮箱 工作职责 绩效管理部 韩杉 李飞 I902 041 人力资源 绩效专员 13800000041 hanshan@educompany.com 建立并维护员工绩效档案，定期组织绩效评价会议，协调各部门反馈，制定考核流程与标准，确保绩效考核的有效执行与公正性。 绩效管理部 李勇 李飞 I903 042 人力资源 绩效专员 13800000042 liyong@educompany.com 协助高层教师的专业成长与提升，参与绩效目标的制定，监控绩效实施过程，撰写绩效趋势分析报告，为决策提供支持。 绩效管理部 曹平 李飞 I904 043 人力资源 绩效专员 13800000043 caoping@educompany.com 负责对绩效考核结果进行分析，撰写改进报告，研究绩效管理的前沿理论与实践，协助各部门制定职业发展与提升计划。 绩效管理部 王斌 李飞 I905 044 人力资源 绩效专员 13800000044 wangbin@educompany.com 定期收集和汇总员工绩效数据，撰写数据分析报告，为后续改进提供依据，协助组织绩效反馈会，确保沟通的畅通与及时。 
</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
| 部门 | 员工姓名 | 员工主管 | 工位 | 工号 | 岗位 | 职位 | 电话 | 邮箱 | 工作职责 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 绩效管理部 | 韩杉 | 李飞 | I902 | 041 | 人力资源 | 绩效专员 | 13800000041 | hanshan@educompany.com | 建立并维护员工绩效档案，定期组织绩效评价会议，协调各部门反馈，制定考核流程与标准，确保绩效考核的有效执行与公正性。 |
| 绩效管理部 | 李勇 | 李飞 | I903 | 042 | 人力资源 | 绩效专员 | 13800000042 | liyong@educompany.com | 协助高层教师的专业成长与提升，参与绩效目标的制定，监控绩效实施过程，撰写绩效趋势分析报告，为决策提供支持。 |
| 绩效管理部 | 曹平 | 李飞 | I904 | 043 | 人力资源 | 绩效专员 | 13800000043 | caoping@educompany.com | 负责对绩效考核结果进行分析，撰写改进报告，研究绩效管理的前沿理论与实践，协助各部门制定职业发展与提升计划。 |
| 绩效管理部 | 王斌 | 李飞 | I905 | 044 | 人力资源 | 绩效专员 | 13800000044 | wangbin@educompany.com | 定期收集和汇总员工绩效数据，撰写数据分析报告，为后续改进提供依据，协助组织绩效反馈会，确保沟通的畅通与及时。 |
``` 
</td>
</tr>
</tbody>
</table>


*   **图片**：图片解析后的信息，加入解析后的文本中，例如：
    

<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>

```markdown
**现象** 
**处理方式** 
1. 查看出问题的pod日志: kubectl logs content-pod -n content-namespace 
2. 检查pod状态: kubectl describe pod content-pod -n content-namespace 
3. 如果需要，可以删除并重启pod: kubectl delete pod content-pod -n content-namespace
```

</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
**现象**

2023-10-20T12:00:00.000Z ERROR PodMonitoring Pod "content-pod" in namespace "content-namespace" is in a CrashLoopBackOff state.

2023-10-20T12:00:00.000Z WARNING PodManager The container "content-container" for pod "content-pod" terminated with exit code 1.

2023-10-20T12:00:05.000Z INFO PodWatcher Restarting container "content-container" in pod "content-pod" (attempt 1 of 5)

2023-10-20T12:00:10.0002 ERROR PodMonitoring Crash loop detected, pod "content-pod" has restarted 5 times in the last 5 minutes.

...(编者按：此处省略其他日志)

**处理方式**

1. 查看出问题的pod日志:
    
    kubectl logs content-pod -n content-namespace
    
2. 检查pod状态:
    
    kubectl describe pod content-pod -n content-namespace
    
3. 如果需要，可以删除并重启pod:
    
    kubectl delete pod content-pod -n content-namespace

``` 
</td>
</tr>
</tbody>
</table>



<style>
    table {
      width: 80%;
      margin: 20px auto; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
### 2.2. 优化文档切片方法

**实现方法**：在我们的项目代码中，我们可以将系统默认的文档切片方式`index = VectorStoreIndex.from_documents(documents)`修改为`index = VectorStoreIndex(nodes)`，通过调用我们自定义的切片工具将数据传入 nodes ，从而实现切片方式改造。

**文档切片优化整体思路**：
优化文档切片方式，整体遵循按markdown文档结构基于语义切片的思路。对于独立的没有明确含义的内容（标题、段落、单元格内容），在切片时补充语义信息到切片中，让信息更加独立和完整。

具体思路：

*   **文本内容，标题进行扩写**：文档内的标题扩写加上各级标题语义。例如内容开发工程师岗位指导说明书文档里的“内容开发工程师>工作流程指导>评审”标题下的“注意事项”标题：
        
<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>

```markdown
...
注意事项
...
```
</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
...
内容开发工程师>工作流程指导>评审>注意事项
...
``` 
</td>
</tr>
</tbody>
</table>



* **文本内容，段落文本带标题一起整体切割**：段落的文本与对应的标题作为一个整体，切片处理时一起放进片段，如果放不下那就一起放到下一个片段。例如对于已经扩写的“注意事项”标题与段落内容：
    
<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>

```markdown
chunk1:
...
内容开发工程师>工作流程指导>评审>注意事项
    
chunk2:
•	确保有多位审阅者参与，提供不同视角的反馈。
•	建立明确的评审标准，方便审阅者进行评价。
•	记录审阅意见，并及时跟进更改。
...
```
</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
chunk1:
...

chunk2:
内容开发工程师>工作流程指导>评审>注意事项
•	确保有多位审阅者参与，提供不同视角的反馈。
•	建立明确的评审标准，方便审阅者进行评价。
•	记录审阅意见，并及时跟进更改。
...
``` 
</td>
</tr>
</tbody>
</table>


* **表格内容，单元格内容进行扩写**：针对每一行单元格的数据，扩写加上各级标题、表格头部的字段说明。例如 IT 部张伟的个人信息：

<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>

```markdown
...
IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置与维护，监控系统运⾏状态，及时处理技术问题与故障，提供技术⽀持及⼯具使⽤培训。
...
```
</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
...
公司各部门职责与关键角色联系方式>各部门关键角色联系人
部门：IT部 
员工姓名：张伟
员工主管：⻢云
工位：H802
工号：036
岗位：IT⽀撑
职位：IT专员
电话：13800000036
邮箱：zhangwei036@educompany.com
工作职责：进⾏公司⽹络及硬件设备的配置与维护，监控系统运⾏状态，及时处理技术问题与故障，提供技术⽀持及⼯具使⽤培训。
...
``` 
</td>
</tr>
</tbody>
</table>


* **表格内容，每一行数据一起整体切割**：表格的每一行数据作为一个整体，切片处理时一起放进同一个片段，如果放不下那就一起放到下一个片段。例如对于已经扩写的 IT 部张伟的个人信息：

<table  width="1000px" align="center">
<thead>
<tr>
    <th width="80px">优化阶段</th>
    <th >提示词</th>
</tr>
</thead>
<tbody>
<tr>
    <td>优化前</td>
<td>

```markdown
chunk1:
...
公司各部门职责与关键角色联系方式>各部门关键角色联系人
部门：IT部 
员工姓名：张伟
员工主管：⻢云
工位：H802
工号：036
岗位：IT⽀撑
职位：IT专员

chunk2:
电话：13800000036
邮箱：zhangwei036@educompany.com
工作职责：进⾏公司⽹络及硬件设备的配置与维护，监控系统运⾏状态，及时处理技术问题与故障，提供技术⽀持及⼯具使⽤培训。
...
```
</td>
</tr>
<tr>
    <td>优化后 </td>
<td>

```markdown
chunk1:
...
    
chunk2:
公司各部门职责与关键角色联系方式>各部门关键角色联系人
部门：IT部 
员工姓名：张伟
员工主管：⻢云
工位：H802
工号：036
岗位：IT⽀撑
职位：IT专员
电话：13800000036
邮箱：zhangwei036@educompany.com
工作职责：进⾏公司⽹络及硬件设备的配置与维护，监控系统运⾏状态，及时处理技术问题与故障，提供技术⽀持及⼯具使⽤培训。
...
``` 
</td>
</tr>
</tbody>
</table>




### 2.3. 优化检索过程
有时你可能会发现，RAG的输出虽然无误，但是缺少信息，比如提问：“我们公司都有哪些部门？”

In [8]:
# 加载百炼的 API Key 用于调用通义千问大模型
from config.load_key import load_key
load_key()

from chatbot import rag
# 上一章节已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
query_engine = rag.create_query_engine(rag.load_index())
rag.ask('我们公司都有哪些部门？', query_engine=query_engine)

我们公司包含的部门有IT部和人力资源部。这些部门负责不同的职能，如IT支撑和人力资源政策的实施等。

从知识库文件可以看到，答疑机器人的回复漏掉了许多部门。

<img src="https://img.alicdn.com/imgextra/i1/O1CN01tj3C3a1KgRVe9ByKY_!!6000000001193-0-tps-1738-860.jpg" alt="知识库文件" width="300px">

这是由于在检索阶段提供给答疑机器人的参考信息不够。为了解决这一问题，我们可以通过以下方法来优化检索过程：

- 选择召回文本段个数
- 使用ReRank和相似度阈值
- 尝试进阶的检索方法：句子滑窗检索与自动合并检索

## 🛠️ 3. 尝试优化知识库问答的能力

通过前面优化思路的分析，可以看到如果考虑更多的文件内容格式的复杂性，具体实现起来是相对复杂的，代码逻辑繁琐、开发效率较低。

你此前已经学习过[阿里云大模型工程师ACA认证课程](https://edu.aliyun.com/course/3126500)，知道如何利用大模型来提升开发效率，因此你想到利用大模型驱动来辅助 RAG 优化。

你了解到[阿里云大模型服务平台百炼](https://help.aliyun.com/zh/model-studio/getting-started/what-is-model-studio)提供了一些有用的工具或方法，可以帮助你完成目标：

*   [DashScopeParse](https://help.aliyun.com/zh/model-studio/developer-reference/dashscopeparse)：基于 LlamaIndex框架的文件解析器，能够将常见的 doc/docx/pdf 文件解析成大模型易于处理的 markdown 文本，并且能够解析文档里的图片信息。你可以将它无缝集成至课程 LlamaIndex项目中，最关键的是，这个接口是免费的。
    
*   [通义千问API](https://help.aliyun.com/zh/model-studio/developer-reference/use-qwen-by-calling-api)：使用文本生成、视觉理解模型辅助进行内容优化。
    

因此，处理方案可以优化成这样：


<div align="center">
<img src="https://gw.alicdn.com/imgextra/i2/O1CN01YXGWYr24XFt4m5yze_!!6000000007400-0-tps-2294-254.jpg" alt="工作流编排" width="80%">
</div>

**处理过程：**

1.  文档解析：将原始文件，经过DashScopeParse 进行处理，将返回的数据处理成初始的 markdown 文本。
    
2.  深层次解析图片： DashScopeParse 处理时，会将文档图片内的文本信息也解析到初始 markdown 文本中（类似 OCR），这对于一些命令行截图、文本截图是足够的，但是对于一些不规则、复杂信息的图片，如果需要深层次理解图片内容，则需要用视觉模型单独处理一次。
    
3.  润色目录层级等：由于 PDF 等文档的结构不规则、格式差异，解析目录时存在目录层级语义、表格逻辑无法解析出的情况，例如标题无法判定是几级标题时系统会预置为一级标题，此时可以用文本模型处理一次文本补全 markdown 目录层级等信息。这一步也是后续做标题改写、表格单元格改写的基础。
    
4.  标题改写：补全目录层级信息，增强标题的独立性与完整性，保证语义信息。
    
5.  表格单元格改写：kv 形式补充单元格信息，增强每行数据以及单元格独立性与完整性，保证语义信息。
    
6.  语义化分割：基于段落为主体进行语义化分割，并保证关键数据的语义独立性与完整性，例如标题下的文本、表格的每行数据等。
    

**为什么不使用一个大模型或者仅调用一次大模型完成所有的处理？**

在 ACA 的课程中我们学习过 Multi-Agent 的概念，这里采用了类似的处理思想。

大模型具有很强大的处理能力，但是如果一个很复杂的任务需求全部由一个模型一次调用来处理，难以同时保证每项工作的质量。

因此，我们可以将复杂的任务需求拆分成不同的子任务，每个子任务交由不同的 agent 去执行。单个 agent 可以根据任务特点，指定不同的模型类型、需求 prompt、模型配置参数，以组合获得最优的整体工作效果。对于一些场景固定、处理简单、开发效率足够支撑业务需求的子任务，你甚至可以自行实现固定的代码逻辑替代对应处理的agent。每个环节具体如何处理，需要根据实际项目复杂度、效率和成本等方面综合考虑。





由于教程中有多处需要调用大模型API，因此我们在这里进行了辅助函数的封装。需要时直接调用辅助函数即可。

In [6]:
from http import HTTPStatus
from dashscope import Generation
import dashscope
import os

# 调用视觉llm实现深度解析与理解图片的内容
def parse_image_to_text(image_path):
    messages = [
        {
            "role": "user",
            "content": [
                {"image": "" + str(image_path)},
                {"text": "解析如下图片的内容。输出三部分：1、图片里面的文本；2、图片出现的场景；3、如果图片中有标红等标识重点，请描述图片重点信息"}
            ]
        }
    ]

    response = dashscope.MultiModalConversation.call(
        model='qwen-vl-max',
        messages=messages,
        api_key=os.getenv('DASHSCOPE_API_KEY'),
    )

    if response.status_code == HTTPStatus.OK:
        return (response.output.choices[0].message.content[0]["text"])
    else:
        return (response.code + "\n" + response.message)


# 文本llm，专注RAG优化
def llm_text_tool(prompt, llm_model):
    messages = [
        {'role': 'system', 'content': '你是一个大模型RAG优化专家，精通利用大模型做RAG优化'},
        {'role': 'user', 'content': prompt}
    ]
    response = Generation.call(
        model=llm_model,
        messages=messages,
        result_format='message'
    )

    return (response.output.choices[0].message.content)

# 调用示例
# 调用文本llm
print(llm_text_tool("大模型RAG是用来做什么的", "qwen-max"))
# 调用视觉llm实现深度解析与理解图片的内容
print(parse_image_to_text("https://gw.alicdn.com/imgextra/i2/O1CN01bURFG01wp6O9WXKeI_!!6000000006356-0-tps-948-143.jpg"))



大模型RAG（Retrieval-Augmented Generation）是一种结合了信息检索和文本生成的技术，旨在提高大型语言模型的性能，尤其是在处理需要外部知识的任务时。RAG的主要用途包括但不限于以下几个方面：

1. **增强事实准确性**：通过从一个大规模的知识库或文档集合中检索相关信息，然后将这些信息融入到生成的回答中，RAG能够帮助提高回答的事实准确性和丰富度。这对于那些要求高度准确性的应用场景尤其重要，比如学术研究、医疗咨询等。

2. **支持长尾问题解决**：对于一些非常具体或者不常见的问题（即所谓的“长尾”问题），传统的基于模式识别的方法可能难以给出满意答案。而RAG可以通过搜索相关文献或数据来提供更精准的答案，从而更好地服务于这类需求。

3. **改善对话系统的交互体验**：在构建聊天机器人或者其他形式的对话系统时，使用RAG可以让机器更加自然流畅地与人类用户交流，并且能够根据上下文灵活调整其响应内容，使得整个对话过程更加贴近真实的人际沟通。

4. **促进跨领域知识应用**：由于RAG允许访问广泛领域的外部资源，因此它可以帮助跨越不同学科之间的界限，在多个专业领域内实现知识的有效整合与利用，为用户提供更为全面的服务。

总之，RAG技术通过融合检索技术和生成模型的优势，不仅提升了现有语言模型处理复杂任务的能力，也为开发更加智能、可靠的信息服务提供了新的可能性。
### 图片里面的文本

```
2023-10-28T12:00:00Z ERROR PodMonitoring Pod "content-pod" in namespace "content-namespace" is in a CrashLoopBackOff state.
2023-10-28T12:00:01Z WARNING PodManager The container "content-container" for pod "content-pod" terminated with exit code 1.
2023-10-28T12:00:01Z INFO PodWatcher Restarting container "content-container" in pod "content-pod" (attempt 1 of 5)
2023-10-28T1

### 3.1. 文档解析

针对文档解析环节，这里我们以[DashScopeParse](https://help.aliyun.com/zh/model-studio/developer-reference/dashscopeparse)处理进行说明。DashScopeParse能够将常见的 doc/docx/pdf 等文件格式解析成大模型易于处理的 markdown 文本，并且能够解析文档里的图片信息。你可以将它无缝集成至课程 LlamaIndex项目中，最关键的是，这个接口是免费的。

实际业务中，如果文档类型格式与场景固定、处理简单、开发效率足够支撑业务需求，你可以自行实现固定的代码逻辑替代该处理环节。
> 你需要更改倒数第三行中category_id的值。

In [ ]:
from llama_index.readers.dashscope.utils import ResultType
from llama_index.readers.dashscope.base import DashScopeParse
from dashscope import Generation

import os
import re
import json
import nest_asyncio

nest_asyncio.apply()
# 使用环境变量
os.environ['DASHSCOPE_API_KEY'] = os.getenv('DASHSCOPE_API_KEY')



# 由于pdf/docx等多种文件格式来源的多样性，文件解析到markdown过程中可能有一定损耗。使用llm对生成的markdown文本进行润色，修正目录层级、缺失信息等
def md_polisher(data):
    messages = [
        {'role': 'user', 'content': '下面这段文本是由pdf转为markdown的，格式和内容可能存在一些问题，需要你帮我优化下：1、目录层级，如果目录层级顺序不对请以markdown形式补全或修改；2、内容错误，如果存在上下文不一致的情况，请你修改下；3、如果有表格，注意上下行不一致的情况；4、输出文本整体应该与输入没有较大差异，不要自己制造内容，我是需要对原文进行润色；4、输出格式要求：markdown文本，你的所有回答都应该放在一个markdown文件里面。特别注意：不需要多余的说明'+data}
        ]
    response = Generation.call(
        model="qwen-plus",
        messages=messages,
        result_format='message'
    )

    return(response.output.choices[0].message.content)


# 文件通过DashScopeParse接口解析为程序与大模型易于处理的markdown文本
def file_to_md(category_id,file):
    parse = DashScopeParse(
        result_type=ResultType.DASHSCOPE_DOCMIND,
        category_id=category_id
    )
    documents = parse.load_data(file_path=file)
    # 初始化一个空字符串来存储Markdown内容
    markdown_content = ""
    for doc in documents:
        doc_json = json.loads(json.loads(doc.text))
        for item in doc_json["layouts"]:
            if item["text"] in item["markdownContent"]:
                markdown_content += item["markdownContent"]
            else:
                # DashScopeParse处理时，会将文档图片内的文本信息也解析到初始markdown文本中（类似OCR），这对于本文示例文件中的命令行截图、文本截图是足够的，示例无需深层次解析图片。
                # 实际场景中的知识库文档，如果涉及不规则、复杂信息的图片并且需要深层次理解图片内容，您可以调用视觉模型进一步理解图片含义。
                # （DashScopeParse返回的数据结构中，针对图片数据，markdownContent字段是图片url，text字段是解析出的文本）
                # if ".jpg" in item["markdownContent"] or ".jpeg" in item["markdownContent"] or ".png" in item["markdownContent"]:
                #     image_url = re.findall(r'\!\[.*?\]\((https?://.*?)\)', item["markdownContent"])[0]
                #     print(image_url)
                #     markdown_content = markdown_content + parse_image_to_text(image_url)+"\n"
                # else:
                #     markdown_content = markdown_content + item["text"]+"\n"
                markdown_content = markdown_content + item["text"]+"\n"
    print(markdown_content)
    return md_polisher(markdown_content)

### 调用示例

# 1、可选配置。
# 百炼平台上，可以对不同项目配置不同的业务空间，默认情况下是使用默认业务空间。
# 如果需要使用非默认空间，可以前往[百炼控制台-业务空间管理](https://bailian.console.aliyun.com/?admin=1#/efm/business_management)，配置业务空间并获取Workspace ID。
# 完成后，取消注释并修改这段代码为实际值：
# os.environ['DASHSCOPE_WORKSPACE_ID'] = "<Your Workspace id, Default workspace is empty.>"

# 2、必选配置。
# 文件通过DashScopeParse进行解析时，需要配置上传的数据目录id。可以前往[百炼控制台-数据管理](https://bailian.console.aliyun.com/#/data-center)，配置类目并获取ID
category_id="cate_xxx" # 需要修改为您的实际值

md_content = file_to_md(category_id,['./docs/测试文档1.pdf'])
print(md_content)


上述示例代码中仅使用DashScopeParse已经可以满足图片处理需求。

如果需要进行图片深层次解析，可参考如下处理效果示例。

图片输入：

<img src="https://gw.alicdn.com/imgextra/i2/O1CN01bURFG01wp6O9WXKeI_!!6000000006356-0-tps-948-143.jpg" alt="工作流编排" width="600px">



In [7]:
print(parse_image_to_text("https://gw.alicdn.com/imgextra/i2/O1CN01bURFG01wp6O9WXKeI_!!6000000006356-0-tps-948-143.jpg"))

### 图片内容
```
2023-10-28T12:00:00Z ERROR PodMonitoring Pod "content-pod" in namespace "content-namespace" is in a CrashLoopBackOff state.
2023-10-28T12:00:01Z WARNING PodManager The container "content-container" for pod "content-pod" terminated with exit code 1.
2023-10-28T12:00:01Z INFO PodWatcher Restarting container "content-container" in pod "content-pod" (attempt 1 of 5)
2023-10-28T12:00:01Z ERROR PodMonitoring Crash loop detected, pod "content-pod" has restarted 5 times in the last 5 minutes.
2023-10-28T12:00:01Z INFO PodManager Warning: BackOff: Back-off restarting failed container.
2023-10-28T12:00:01Z INFO EventRecorder Warning: BackOff: Back-off restarting failed container.
2023-10-28T12:00:01Z ERROR PodWatcher Investigating logs for pod "content-pod" for error diagnosis.
2023-10-28T12:00:01Z ERROR PodWatcher Error: logs from container "content-pod" container not found.
2023-10-28T12:00:01Z INFO PodCondition Pod "content-pod" transitioning status from Running to CrashLoopBackO

### 3.2. 文档切片

#### 3.2.1 标题改写

这里我们使用大模型帮助我们进行标题改写。

如果处理逻辑固定、处理简单、开发效率足够支撑业务需求，你也可以自行实现固定的代码逻辑替代该处理环节。

In [13]:
def title_expander(content):
    prompt = """
    下面这段文本是markdown格式的，需要你帮我扩写下标题（即只替换标题 不修改其他内容）：
    1、标题扩写规则：
    如果是一级标题，则不用扩写
    如果是二级标题，则标题替换为一级标题+二级标题，格式：一级标题>二级标题
    如果是三级标题，则标题替换为一级标题+二级标题+三级标题，格式：一级标题>二级标题>三级标题
    ...以此类推
    1.2、标题扩写示例：
    原始文本：
    # 一级标题
    一级标题下的文本
    ## 二级标题
    二级标题下的文本
    ### 三级标题
    三级标题下的文本
    扩写后：
    # 一级标题
    一级标题下的文本
    ## 一级标题>二级标题
    二级标题下的文本
    ### 一级标题>二级标题>三级标题
    三级标题下的文本

    2、输出格式要求：markdown格式，你的所有回答都应该放在一个可直接使用的markdown纯文本里面。
    注意：不要加其他多余内容，去除开头的“```markdown”和末尾的“```
    如下是文本：\n"""+content

    return (llm_text_tool(prompt, "qwen-plus"))

# 调用示例
title_expander_content = title_expander(md_content)
print(title_expander_content)

# 内容开发工程师

## 内容开发工程师>岗位类型

- 大类: 技术大类
- 细分类型: 综合技术岗位

## 内容开发工程师>工作职责

### 内容开发工程师>工作职责>核心职责

结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。

### 内容开发工程师>工作职责>详细职责

1. **内容研究与分析**
   - 对最新的教育技术趋势、学习理论和市场需求进行深入研究。
   - 这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。
   - 通过持续的市场调研，确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。

2. **教材和课程开发**
   - 根据研究和市场反馈，设计和开发高质量的教育教材和课程。
   - 这包括撰写教学大纲、制作课件、设计评估工具等。
   - 确保内容符合教育标准和学习目标，以提供全面的学习体验。
   - 考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。

3. **内容优化与更新**
   - 在内容开发过程中，不断优化已有的教育材料。
   - 通过跟踪学习者的反馈和评价，识别出内容中的潜在问题，并及时进行调整。
   - 定期更新材料，以反映新的研究成果、技术进步和市场变化。
   - 保持内容的时效性和相关性是重要任务之一。


#### 3.2.2 表格改写

这里我们使用大模型帮助我们进行表格改写。

如果处理逻辑固定、处理简单、开发效率足够支撑业务需求，你也可以自行实现固定的代码逻辑替代该处理环节。

In [11]:
def table_expander(content):
    prompt = """
    下面这段文本是markdown格式的，需要你帮我改写表格信息（即只作替换 不修改其他内容）：
    1、表格扩写规则：针对表格，将每行数据，转化为kv形式字典，示例：
    [
    {"部门"：“教研部”,"员工姓名":"张伟",...},
    {...}
    ]
    2、非表格内容不要改动。
    注意：如果输入信息中没有表格，那么不要做任何处理，直接将输入信息作为输出信息原样返回。
    3、输出格式要求：markdown格式，你的所有回答都应该放在一个可直接使用的markdown纯文本里面。
    注意：不要加其他多余内容，去除开头的“```markdown”和末尾的“```
    如下是文本：\n"""+content

    return (llm_text_tool(prompt, "qwen-plus"))

# 调用示例

test_table = """
...
## 公司各部门职责与关键角色联系方式>各部门关键角色联系人
...
| 部门   | 员工姓名 | 员工主管 | 工位   | 工号  | 岗位    | 职位     | 电话          | 邮箱                 | 工作职责                                                                                                                                                                                                                           |
| ------ | -------- | -------- | ------ | ----- | ------- | -------- | -------------- | -------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| 教研部 | 张伟     | 李琳     | A101   | 001   | 内容设计 | 教研专员 | 13800000001    | zhangwei@educompany.com | 负责教育课程的研究与开发，分析教学效果，整理教案，协助课程优化，以及参与教育项目的评估和反馈。                                                                                                                                                   |
| 教研部 | 王芳     | 李琳     | A102   | 002   | 内容设计 | 教研专员 | 13800000002    | wangfang@educompany.com | 负责制定学科教学方案，策划教学活动，编写教案，收集学生反馈，参与课程改进会议，提供专业意见。                                                                                                                                                     |
| 教研部 | 刘杰     | 李琳     | A103   | 003   | 内容设计 | 教研专员 | 13800000003    | liujie@educompany.com   | 组织教研活动，分析教学成果，开展学术研究，协助撰写研究报告，输出有效教学策略，提升课程品质。                                                                                                                                                       |
| 教研部 | 陈敏     | 李琳     | A104   | 004   | 内容设计 | 教研专员 | 13800000004    | chenmin@educompany.com  | 参与教研小组讨论，整理教学材料，进行课程评估，提出创新教学方案，收集并分析测评数据以指导教学调整。                                                                                                                                                     |
| 教研部 | 杨洋     | 李琳     | A105   | 005   | 内容设计 | 教研专员 | 13800000005    | yangyang@educompany.com | 协助制定年度教研计划，组织教师培训，实施教室观察，编写教研活动总结，提供教学建议，提升教师专业发展。                                                                                                                                                |
| 教研部 | 赵伟     | 李琳     | A106   | 006   | 内容设计 | 教研专员 | 13800000006    | zhaowei@educompany.com  | 负责搜集教育前沿信息，撰写分析报告，参与教育技术的研究，设计教具，并提供相应的使用培训和支持。                                                                                                                                                         |
| 课程开发部 | 李丽 | 王强 | B201 | 007 | 内容开发 | 课程开发专员 | 13800000007 | lili@educompany.com | 设计并开发教育课程，撰写课程大纲，组织课程试点，分析市场需求，调整课程内容，以适应学习者的需求和反馈。                                                                                                                                             |
| 课程开发部 | 张伟 | 王强 | B202 | 008 | 内容开发 | 课程开发专员 | 13800000008 | zhangwei01@educompany.com | 精细化课程设计，确保课程符合国家教育标准，撰写教具使用说明，协助教研部进行课程的前期调研与评估。                                                                                                                                                   |
| 课程开发部 | 郑强 | 王强 | B203 | 009 | 内容开发 | 课程开发专员 | 13800000009 | zhengqiang@educompany.com | 进行课程实施效果评估，定期收集课程反馈，参与课程迭代优化，并与教学团队沟通课程实施问题，汇报调整建议。                                                                                                                                             |
| 课程开发部 | 吴昊 | 王强 | B204 | 010 | 内容开发 | 课程开发专员 | 13800000010 | wuh@educompany.com | 协助教学内容的整合与数字化，开发辅助学习资源，调研新技术在课程中的应用，撰写课程评估报告，提出改进方案。                                                                                                                                           |
| 课程开发部 | 周丹 | 王强 | B205 | 011 | 内容开发 | 课程开发专员 | 13800000011 | zhou@educompany.com | 负责课程相关资料的管理及更新，优化课程展示形式，向教师提供课程内容培训，解答教学实施中的疑问与挑战。                                                                                                                                               |
| 教材编写部 | 孙杰 | 陈刚 | C301 | 012 | 内容开发 | 教材编写专员 | 13800000012 | sunjie@educompany.com | 负责教材的编写与修订，收集教材使用反馈，开展教材审稿，确保教材内容的准确性与教育性，进行编写方案的评估与调整。                                                                                                                                     |
"""

table_expander_content = table_expander(test_table)
print(table_expander_content)

...
## 公司各部门职责与关键角色联系方式>各部门关键角色联系人
...
[
{"部门":"教研部","员工姓名":"张伟","员工主管":"李琳","工位":"A101","工号":"001","岗位":"内容设计","职位":"教研专员","电话":"13800000001","邮箱":"zhangwei@educompany.com","工作职责":"负责教育课程的研究与开发，分析教学效果，整理教案，协助课程优化，以及参与教育项目的评估和反馈。"},
{"部门":"教研部","员工姓名":"王芳","员工主管":"李琳","工位":"A102","工号":"002","岗位":"内容设计","职位":"教研专员","电话":"13800000002","邮箱":"wangfang@educompany.com","工作职责":"负责制定学科教学方案，策划教学活动，编写教案，收集学生反馈，参与课程改进会议，提供专业意见。"},
{"部门":"教研部","员工姓名":"刘杰","员工主管":"李琳","工位":"A103","工号":"003","岗位":"内容设计","职位":"教研专员","电话":"13800000003","邮箱":"liujie@educompany.com","工作职责":"组织教研活动，分析教学成果，开展学术研究，协助撰写研究报告，输出有效教学策略，提升课程品质。"},
{"部门":"教研部","员工姓名":"陈敏","员工主管":"李琳","工位":"A104","工号":"004","岗位":"内容设计","职位":"教研专员","电话":"13800000004","邮箱":"chenmin@educompany.com","工作职责":"参与教研小组讨论，整理教学材料，进行课程评估，提出创新教学方案，收集并分析测评数据以指导教学调整。"},
{"部门":"教研部","员工姓名":"杨洋","员工主管":"李琳","工位":"A105","工号":"005","岗位":"内容设计","职位":"教研专员","电话":"13800000005","邮箱":"yangyang@educompany.com","工作职责":"协助制定年度教研计划，组织教师培训，实施教室观察，编写教研活动总结，提供教学建议，提升教师专业发展。"},

#### 3.2.3 分割文本

这里我们使用大模型帮助我们进行分割文本。

如果处理逻辑固定、处理简单、开发效率足够支撑业务需求，你也可以自行实现固定的代码逻辑替代该处理环节。

In [12]:
def para_spliter(content):
    prompt = """
    下面这段文本是markdown格式的，需要你帮我做一下切片：
    1、切片规则：
    1.1、基于语义切片，你的所有的切分，截断后不要丢失文字上下文原本的语义
    1.2、如果标题下有内容，则内容与标题作为整体，放在同一个元素中，保证语义的完整性，不要拆开
    示例：
    ...
    ### 三级标题
    三级标题下的文本
    ...
    1.3、如果标题下没有文本，则合并到标题合并到下一个元素
    示例，下面的二级标题下没有文本，所以合并到下一个元素中，作为一个切片元素
    ...
    ## 二级标题
    ### 三级标题
    三级标题下的文本
    ...
    1.4、如果切片长度过短，需要考虑切片合并，原则上，在同一个大标题下的切片进行合并，但是合并后不要超过512字符
    示例，下面这种情况需要切片合并：
     -合并前：
    chunk1:（假设长度远小于512字符）
    ### 三级标题1
    三级标题1下的文本
    chunk2:（假设长度远小于512字符）
    ### 三级标题2
    三级标题2下的文本
     -合并后：是在同一个chunk中：
    ...
    ### 三级标题1
    三级标题1下的文本
    ### 三级标题2
    三级标题2下的文本
    ...
    1.4、如果标题下的文本过长（超过512字符视为过长），则需要拆分为多个切片元素。
    拆分要求1:带标题拆分
    拆分要求2:拆分后的片段，标题带上片段顺序
    示例：
    chunk1:
    ### 三级标题-片段1
    三级标题下的文本第一部分
    chunk2:
    ### 三级标题-片段2
    三级标题下的文本第二部分
    chunk3:
    ### 三级标题-片段3
    三级标题下的文本第三部分
    2、表格json数据的切片：下面这种处理后的表格的每一行数据作为一个整体，切片处理时一起放进chunk，如果放不下那就一起放到下一个chunk
    示例：
    {"部门":"教研部","员工姓名":"张伟","员工主管":"李琳","工位":"A101","工号":"001","岗位":"内","职位":"教研专员","电话":"13800000001","邮箱":"zhangwei@educompany.com","工作职责":"负责教育课程的研究与开发，分析教学效果，整理教案，协助课程优化，以及参与教育项目的评估和反馈。"}
    注意事项：如果表格chunk进行了区分，则需要把所属的标题也带进去,保证语义完整性；数据条目不要有变化
    3、输出格式要求：python字典形式，我需要可以通过json.loads加载为json处理。注意：你的所有回答不要加其他多余内容，去除开头的“```python”和末尾的“```“
    示例：
    {
    "chunk1":"切分后的元素",
    "chunk2":"切分后的元素",
    "chunk3":"切分后的元素"
    }
    如下是文本：\n"""+content

    return (llm_text_tool(prompt, "qwen-plus"))

# 调用示例
chunkers = para_spliter(title_expander_content)
print(chunkers)


{
    "chunk1": "# 内容开发工程师\n\n## 内容开发工程师>岗位类型\n\n- 大类: 技术大类\n- 细分类型: 综合技术岗位\n\n## 内容开发工程师>工作职责\n\n### 内容开发工程师>工作职责>核心职责\n\n结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。",
    "chunk2": "### 内容开发工程师>工作职责>详细职责\n\n1. **内容研究与分析**\n   - 对最新的教育技术趋势、学习理论和市场需求进行深入研究。\n   - 这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。\n   - 通过持续的市场调研，确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。",
    "chunk3": "2. **教材和课程开发**\n   - 根据研究和市场反馈，设计和开发高质量的教育教材和课程。\n   - 这包括撰写教学大纲、制作课件、设计评估工具等。\n   - 确保内容符合教育标准和学习目标，以提供全面的学习体验。\n   - 考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。",
    "chunk4": "3. **内容优化与更新**\n   - 在内容开发过程中，不断优化已有的教育材料。\n   - 通过跟踪学习者的反馈和评价，识别出内容中的潜在问题，并及时进行调整。\n   - 定期更新材料，以反映新的研究成果、技术进步和市场变化。\n   - 保持内容的时效性和相关性是重要任务之一。"
}


### 3.3 检索过程优化
我们可以通过以下方法来优化检索过程：

- 选择召回文本段个数
- 使用ReRank和相似度阈值
- 尝试进阶的检索方法：句子滑窗检索与自动合并检索

#### 3.3.1 修改召回文本段个数
当用户的提问可能涉及到多个知识点时，你设计的答疑机器人可能会输出不完整的信息。为了解决这个问题，你可以选择召回更多的文本段，来为答疑机器人提供更全面的参考信息。

在as_query_engine方法里，通过设定similarity_top_k参数来修改召回文本段的个数。

我们建议你多尝试几个值，并通过2.6的RAGAS评测方案来评测效果，找到最适合的值。

In [24]:
from chatbot.rag import load_index,update_prompt_template
from llama_index.llms.dashscope import DashScope

# 从本地加载索引
index = load_index()
query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus"),
    # 设置召回文本段个数
    similarity_top_k=14,
    streaming=True
)
# 更新提示词模板
query_engine = update_prompt_template(query_engine)
response = query_engine.query("我们公司都有哪些部门")
print("正在生成回复...")
response.print_response_stream()
print("\n")
print(f"召回文本段个数：{len(response.source_nodes)}")

提示词模板修改成功
正在生成回复...
我们公司的部门有：教研部、课程开发部、教材编写部、评估部、市场部、人力资源部、IT部、绩效管理部。

召回文本段个数：14


可以看到，similarity_top_k设置较大值时，答疑机器人输出了完整的结果。

你可以通过答疑机器人返回**response**中包含的source_nodes属性，查看召回文本段的内容以及它与用户提问的语义相似度。LlamaIndex官方提供了display_source_node方法，可以美观地展示source_nodes内容。

In [25]:
from llama_index.core.response.notebook_utils import display_source_node

def show_chunk_score(response):
    for i in range(len(response.source_nodes)):
        print("="*100)
        source_node = response.source_nodes[i]
        display_source_node(source_node=source_node,source_length=300)

show_chunk_score(response)
# 在输出中，text为召回的文本段，Similarity为文本段与用户输入的相似度

**Node ID:** 3a88a80b-3dfa-4e6a-9649-1461be6d0a9e<br>**Similarity:** 0.29068594298673855<br>**Text:** 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 
IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置<br>

**Node ID:** e16d9374-e54b-40f7-ba19-63374428b9f9<br>**Similarity:** 0.28256496103979706<br>**Text:** 职业发展计划。 
⼈⼒资源部 熊伟 ⻢婷 F604 029 ⼈⼒资源 ⼈⼒资源专员 13800000029 xiongwei@educompany.com 负责公司各项⼈⼒资源政策的实施与执⾏，分析⼈⼒资源数据以优化资源配置，提供决策<br>

**Node ID:** 02189473-a250-413d-ad20-3a60af57f3e0<br>**Similarity:** 0.28034777121327314<br>**Text:** 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 
⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，<br>

**Node ID:** 4827e26e-512a-4eb4-ab4c-ab157c39fa26<br>**Similarity:** 0.2745460497686047<br>**Text:** ⽀持，确保⼈⼒资源部⻔顺畅运作。 
⾏政部 黎晗 蔡静 G704 033 ⾏政 ⾏政专员 13800000033 lih@educompany.com 负责采购办公设备与耗材，登记与管理公司固定资产，协助实施绩效考<br>

**Node ID:** 82460c0f-75ec-429c-bdbc-ef6059931482<br>**Similarity:** 0.26641356621651113<br>**Text:** ⽀持。 
绩效管理部 韩杉 李⻜ I902 041 ⼈⼒资源 绩效专员 13800000041 hanshan@educompany.com 建⽴并维护员⼯绩效档案，定期组织绩效评价会议，协调各部⻔反馈，制定考核流程与标准，确保绩效<br>

...

**Node ID:** 72d055b7-c6bd-4bc7-b99d-bbced2cc87d5<br>**Similarity:** 0.21191143262814824<br>**Text:** 与维护，监控系统运⾏状态，及时处理技术问题与故障，提供技术⽀持及⼯具使⽤培训。 IT部 谢宇 ⻢云 H803 037 IT⽀撑 IT专员 13800000037 xieyu@educompany.com ⽀持公司软件系统<br>

**Node ID:** 5e5b408b-2123-4d3c-83e6-08169bafbdd1<br>**Similarity:** 0.2096664380622192<br>**Text:** 相关⽂档整理，参与评估结果的反馈会议，提供数据⽀持，监督评估实施过程，确保评估的全⾯性与公正性。<br>

**Node ID:** 55298b6e-9092-4f2f-b420-e9b7656563f9<br>**Similarity:** 0.1914883162785519<br>**Text:** 内容公司各部⻔职责与关键⻆⾊联系⽅式 公司定位 我们的公司致⼒于推动教育的数字化转型，专注于开发富有创意和互动性的在线学习平台与⼯具。 我们结合最新的教育理论与技术创新， 旨在提升学习者的参与度和学习效果。 通过提供个性化学习体验， 我们希望满⾜不同年龄段和背景的学习需求， 助⼒教育公平和终⾝学习。我们的使命是使知识获取更加便捷、⾼效，让每个⼈都能充分发挥潜⼒。 各部⻔介绍 教研部：教育课程研究设计 课程开发部：技术内容需求开发 教材编写部：教材、练习题等内容汇编与修订 评估部：内容质量质检 市场部：市场活动营销 ⼈⼒资源部：⼈⼒资源管理 IT部：IT技术⽀撑 绩效管理部：⼈员绩效考核设...<br>

**Node ID:** 9de52dfb-16ac-4747-ad65-ff1fd271ad67<br>**Similarity:** 0.1914196213379756<br>**Text:** 内容开发工程师

岗位类型

大类：技术大类

细分类型：综合技术岗位

工作职责

核心职责

结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。

详细职责

1. 内容研究与分析

对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。

2. 教材和课程开发

根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件...<br>

**Node ID:** ddb10359-0706-491f-995e-dce906d82325<br>**Similarity:** 0.18882785278510567<br>**Text:** 段学⽣的学习需求，撰写教材使⽤报告，积极参与教材的试⽤和反馈收集，促进优化迭代。 评估部 邓琪 张晓 D405 021 内容质检 评估专员 13800000021 dengqi@educompany.com 协助评估项⽬的<br>

#### 3.3.2 使用ReRank和相似度阈值
在通过show_chunk_score方法查看召回文本段时，你发现召回的文本段可能与你提问的问题并不相关。这是因为基于embedding模型的向量化检索虽然能够高效地找到相似的文本片段，但这些结果可能并不完全精确或最优。

你可以通过rerank模型（重排序），在初步检索结果的基础上（一般会先设置比较大的similarity_top_k）进行再排序（通过top_n参数设置最终召回的文本段个数），召回最相关的文本片段。

你也可以通过设置相似度阈值，在已召回的文本段中筛选出相似度高于阈值的文本段，减少无用信息对大模型生成时的干扰。

这两个方法可以在as_query_engine中通过node_postprocessors（后处理模块）参数传入。

In [19]:
from llama_index.postprocessor.dashscope_rerank import DashScopeRerank
from llama_index.core.postprocessor import SimilarityPostprocessor
query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus"),
    # 先设置一个较大的值
    similarity_top_k=20,
    streaming=True,
    node_postprocessors=[
        # 在rerank模型中选择你最终想召回的文本段个数，重排模型选择通义实验室的gte-rerank模型
        DashScopeRerank(top_n=2, model="gte-rerank"),
        # 设置一个相似度阈值，低于该阈值的文本段会被过滤掉
        SimilarityPostprocessor(similarity_cutoff=0.2)])
# 更新提示词模板
query_engine = update_prompt_template(query_engine)
response = query_engine.query("我们公司都有哪些部门")
response.print_response_stream()

提示词模板修改成功


我们公司有教研部、课程开发部、教材编写部、评估部、市场部、人力资源部、IT部、绩效管理部。

我们可以看到，由于rerank的存在，尽管最终只召回了两个片段，但是达到了与rerank前召回14个片段一样的效果。

In [23]:
# 打印出文本段内容与相似度分数
show_chunk_score(response)

**Node ID:** 55298b6e-9092-4f2f-b420-e9b7656563f9<br>**Similarity:** 0.501269314155733<br>**Text:** 内容公司各部⻔职责与关键⻆⾊联系⽅式 公司定位 我们的公司致⼒于推动教育的数字化转型，专注于开发富有创意和互动性的在线学习平台与⼯具。 我们结合最新的教育理论与技术创新， 旨在提升学习者的参与度和学习效果。 通过提供个性化学习体验， 我们希望满⾜不同年龄段和背景的学习需求， 助⼒教育公平和终⾝学习。我们的使命是使知识获取更加便捷、⾼效，让每个⼈都能充分发挥潜⼒。 各部⻔介绍 教研部：教育课程研究设计 课程开发部：技术内容需求开发 教材编写部：教材、练习题等内容汇编与修订 评估部：内容质量质检 市场部：市场活动营销 ⼈⼒资源部：⼈⼒资源管理 IT部：IT技术⽀撑 绩效管理部：⼈员绩效考核设...<br>

**Node ID:** 3a88a80b-3dfa-4e6a-9649-1461be6d0a9e<br>**Similarity:** 0.3706688383577123<br>**Text:** 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 
IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置<br>

最相关的文本段在rerank后排在了最靠前的位置。而在进行rerank前，该文本段仅排在第12位。

#### 3.3.3 进阶的检索方法

我们会介绍两种进阶的检索方法。如果你想继续提升RAG问答系统的性能，可以尝试以下两种检索方法。

##### 3.3.3.1 句子滑窗检索
句子滑窗检索是一种能够在检索阶段排除噪声干扰信息，在生成阶段补充信息的方法。具体步骤如下：

1. 将文本按照"."、"?"、"!"切分成chunk；
2. 设置window_size参数，构建K-V对。K值为第一步切分的chunk；V值为以该chunk为中心，向左右两侧各寻找window_size个chunk;
3. 在检索时，将用户提问与第二步的K值对比，将检索出的K值对应的V值取出，通过后处理的模块将其作为生成时的参考内容。

总结来说：句子滑窗检索先检索出最相关的句子，然后将该句子的上下文信息作为参考，从而提高生成内容的相关性和准确性。

In [40]:
from llama_index.core.schema import Document
from llama_index.core import SimpleDirectoryReader
from llama_index.core.response.notebook_utils import display_response


def get_documents(path):
    documents = SimpleDirectoryReader(path).load_data()
    document = Document(text="\n\n".join([doc.text for doc in documents]))
    # 将中文的标点符号预先转化为英文标点符号，否则LlamaIndex无法正确按句子进行切分
    replacements = {
        '。': '. ',
        '！': '! ',
        '？': '? '
    }
    for src, dst in replacements.items():
        document.text = document.text.replace(src, dst)
    print("已转换为英文标点符号")
    return document

# 获取docs文件夹中的document
document = get_documents(path="docs")

from llama_index.core.node_parser import SentenceWindowNodeParser
node_parser = SentenceWindowNodeParser.from_defaults(
    # 在这里设置窗口大小，即以该chunk为中心向两侧各扩展3个句子作为参考信息
    window_size=5,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)
nodes = node_parser.get_nodes_from_documents([document])
print("已开启句子滑窗检索")

from llama_index.core.postprocessor import MetadataReplacementPostProcessor

window_text_replace = MetadataReplacementPostProcessor(
    target_metadata_key="window"
)
query_engine = index.as_query_engine(
    llm=DashScope(model_name="qwen-plus"),
    similarity_top_k=11,
    node_postprocessors=[window_text_replace])
response = query_engine.query("我们公司都有哪些部门")
display_response(response)

已转换为英文标点符号
已开启句子滑窗检索


**`Final Response:`** 公司包含多个部门，如IT部、人力资源部、行政部、绩效管理部和市场部等。这些部门各自承担不同的职责，确保公司的正常运营和发展。

In [37]:
show_chunk_score(response)

**Node ID:** 3a88a80b-3dfa-4e6a-9649-1461be6d0a9e<br>**Similarity:** 0.29068594298673855<br>**Text:** 组织公司活动的前期准备与后期评估，确保公司各项⼯作的顺利进⾏。 
IT部 张伟 ⻢云 H802 036 IT⽀撑 IT专员 13800000036 zhangwei036@educompany.com 进⾏公司⽹络及硬件设备的配置<br>

**Node ID:** e16d9374-e54b-40f7-ba19-63374428b9f9<br>**Similarity:** 0.28256496103979706<br>**Text:** 职业发展计划。 
⼈⼒资源部 熊伟 ⻢婷 F604 029 ⼈⼒资源 ⼈⼒资源专员 13800000029 xiongwei@educompany.com 负责公司各项⼈⼒资源政策的实施与执⾏，分析⼈⼒资源数据以优化资源配置，提供决策<br>

**Node ID:** 02189473-a250-413d-ad20-3a60af57f3e0<br>**Similarity:** 0.28034777121327314<br>**Text:** 核，提供⾏政管理与协调⽀持，优化⾏政⼯作流程。 
⾏政部 秦⻜ 蔡静 G705 034 ⾏政 ⾏政专员 13800000034 qinf@educompany.com 维护公司档案与信息系统，负责公司通知及公告的发布，<br>

## 🛠️ 4. 如何把方案集成到项目代码

通过对优化方案中各个处理环节的尝试，你已经了解了文档解析、标题/表格处理等问题对应的处理方法，并完成了各方法的子函数的编写。

接下来，我们将优化方案集成到我们的课程项目中，并检验方案修改后的实际效果。

### 4.1. 日志记录

由于方案中涉及了比较多的处理环节，部分环节可能存在处理失败的情况，为了便于后续的问题排查，我们可以将处理过程中的日志通过统一的格式记录到日志文件中。

我们可以定义统一的配置参数与日志格式（日志格式类似java的springboot日志格式）。在后续调用时，日志除了在终端中打印外，也会以文件形式记录在app.log日志文件中。

In [ ]:

import logging
import logging.config  # 确保运行时该模块被导入


# 定义日志配置
LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    'formatters': {
        'standard': {
            'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        },
    },
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'formatter': 'standard',
            'stream': 'ext://sys.stdout',  # 默认是 sys.stderr
        },
        'file': {
            'class': 'logging.FileHandler',
            'formatter': 'standard',
            'filename': 'app.log',
        },
    },
    'loggers': {
        '': {  # root logger
            'level': 'INFO',
            'handlers': ['console', 'file'],
        },
        'my_module': {
            'level': 'INFO',
            'handlers': ['console', 'file'],
            'propagate': False,
        },
    }
}

# 配置日志
logging.config.dictConfig(LOGGING_CONFIG)

logger = logging.getLogger(__name__)

# 使用
logger.info("日志信息")



您可以检查项目文件夹中的app.log文件是否保存了日志信息。

### 4.2. 封装调用入口

我们可以将优化方案中的各个函数，封装为一个函数入口，方便调用。


In [ ]:

# 说明：文件通过DashScopeParse进行解析时，需要配置上传的数据目录 id，可以前往[百炼控制台-数据管理](https://bailian.console.aliyun.com/#/data-center)，配置类目并获取 ID。
# 如果前面已经配置过，此处无需配置
# category_id="cate_****_****"


def doc_parser(file_path):
    logger.info("开始上传文件：%s",file_path)
    # 打印最终的Markdown内容
    md_content = file_to_md(category_id,[file_path])
    logger.info("解析完成，开始扩展标题")
    logger.info(md_content)
    # 标题扩展
    md_title_expanded = title_expander(md_content)
    logger.info("标题扩展完成，开始扩展表格信息")
    logger.info(md_title_expanded)
    # 表格信息扩展
    md_table_expanded = table_expander(md_title_expanded)
    logger.info("表格信息扩展完成，开始分段")
    logger.info(md_table_expanded)
    # 按段落做语义化切片
    md_para_splited = para_spliter(md_table_expanded)
    logger.info("分段处理完成，开始返回数据")
    logger.info(md_para_splited)

    return(json.loads(md_para_splited))

# 调用示例
print(doc_parser('./docs/测试文档1.pdf'))
# print(doc_parser('./docs/内容公司各部门职责与关键角色联系信息汇总.pdf'))
# print(doc_parser('./docs/内容开发工程师岗位指导说明书.docx'))


### 4.3. 修改项目主目录文件

1. （可选）更新依赖

DashScopeParse的依赖包为`llama-index-readers-dashscope`。安装`requirements.txt`中的依赖时，已经包含了该依赖包。

如果你需要更新DashScopeParse，请执行以下命令：


In [ ]:
! pip install llama-index-readers-dashscope


2. 修改项目主目录文件`create_kb.py`

你可以将项目主目录文件`create_kb.py`中的 `create_unstructured_db` 函数做如下调整，切换文档解析与处理方式。

修改前：

```python
....

# 创建非结构化向量数据库
def create_unstructured_db(db_name:str,label_name:list):
    print(f"知识库名称为：{db_name}，类目名称为：{label_name}")
    if label_name is None:
        gr.Info("没有选择类目")
    elif len(db_name) == 0:
        gr.Info("没有命名知识库")
    # 判断是否存在同名向量数据库
    elif db_name in os.listdir(DB_PATH):
        gr.Info("知识库已存在，请换个名字或删除原来知识库再创建")
    else:
        gr.Info("正在创建知识库，请等待知识库创建成功信息显示后前往RAG问答")
        documents = []
        for label in label_name:
            label_path = os.path.join(UNSTRUCTURED_FILE_PATH,label)
            documents.extend(SimpleDirectoryReader(label_path).load_data())
        index = VectorStoreIndex.from_documents(
            documents
        )
        db_path = os.path.join(DB_PATH,db_name)
        if not os.path.exists(db_path):
            os.mkdir(db_path)
            index.storage_context.persist(db_path)
        elif os.path.exists(db_path):
            pass
        gr.Info("知识库创建成功，可前往RAG问答进行提问")

...
```

修改后：

```python


...

from doc_parser import doc_parser

...

# 创建非结构化向量数据库
def create_unstructured_db(db_name:str,label_name:list):
    print(f"知识库名称为：{db_name}，类目名称为：{label_name}")
    if label_name is None:
        gr.Info("没有选择类目")
    elif len(db_name) == 0:
        gr.Info("没有命名知识库")
    # 判断是否存在同名向量数据库
    elif db_name in os.listdir(DB_PATH):
        gr.Info("知识库已存在，请换个名字或删除原来知识库再创建")
    else:
        gr.Info("正在创建知识库，请等待知识库创建成功信息显示后前往RAG问答")
        # documents = []
        # for label in label_name:
        #     label_path = os.path.join(UNSTRUCTURED_FILE_PATH,label)
        #     documents.extend(SimpleDirectoryReader(label_path).load_data())
        # index = VectorStoreIndex.from_documents(
        #     documents
        # )
        nodes = []
        for label in label_name:
            label_path = os.path.join(UNSTRUCTURED_FILE_PATH,label)
            print(label_path)
            for path in os.listdir(label_path):
                file_path = os.path.join(label_path,path)
                print(file_path)
                doc_content = doc_parser(file_path)
                for key, value in doc_content.items():
                    nodes = nodes + [TextNode(text=value)]
        index = VectorStoreIndex(nodes)
        db_path = os.path.join(DB_PATH,db_name)
        if not os.path.exists(db_path):
            os.mkdir(db_path)
            index.storage_context.persist(db_path)
        elif os.path.exists(db_path):
            pass
        gr.Info("知识库创建成功，可前往RAG问答进行提问")

```

### 4.4. 优化前后的效果对比
<style>
    table {
      width: 80%;
      margin: 20px; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>



|   **现象**    |   **优化前**    |   **优化后**    |
|   --- |   --- | ---   |
|**现象1：大模型无法理解文档里的图片信息**|优化前：无法理解获取图片信息<br><img src="https://gw.alicdn.com/imgextra/i3/O1CN015tZv6M1wKsFUK7we2_!!6000000006290-0-tps-2554-1252.jpg" alt="工作流编排" width="600px">|优化后：准确返回文档里的图片信息，并给出处理建议<br><img src="https://gw.alicdn.com/imgextra/i2/O1CN01tWuVjI1bKJq3tnkZI_!!6000000003446-0-tps-2544-958.jpg" alt="工作流编排" width="600px">|
|**现象 2： 大模型无法理解一段文本内容的详细语义**|优化前：与其他岗位如测试岗的注意事项混淆<br><img src="https://gw.alicdn.com/imgextra/i2/O1CN01HTZkar1xXzCh9PpWN_!!6000000006454-0-tps-2620-1134.jpg" alt="工作流编排" width="600px">|优化后：更准确、具体的、有效的答复，符合用户提问场景<br><img src="https://gw.alicdn.com/imgextra/i4/O1CN019OUtU91FERCluvHTY_!!6000000000455-0-tps-2520-1266.jpg" alt="工作流编排" width="600px">|
|**现象 3： 大模型无法理解表格单元格的详细语义**|优化前：无法识别不同单元格语义<br><img src="https://gw.alicdn.com/imgextra/i2/O1CN012w65dh1eqKMipUPqN_!!6000000003922-0-tps-2608-1142.jpg" alt="工作流编排" width="600px">|优化后：准确理解单元格信息<br><img src="https://gw.alicdn.com/imgextra/i3/O1CN01a9SJiW1KwTMf9rmcT_!!6000000001228-0-tps-2506-1056.jpg" alt="工作流编排" width="600px">|


## 5. 扩展阅读：更多 RAG 优化方法或建议参考

你已经通过一个改进案例，学习了RAG问题基本的发现、优化与解决方法。

由于RAG过程的复杂性，不同处理阶段会出现不同的问题，下面将分享一些常见的问题及处理方法，帮助您更好的了解RAG优化并提升你的大模型应用效果。

### 5.1. 文档解析阶段

文档解析阶段的问题，主要分为2方面：
* 业务层面：原始业务文档质量差，例如包括多种不同文件形式/不同版本类型、在同一种类型中格式不统一等。形式、格式上的混乱加大了解析出错的概率。
* 技术层面：针对不同类型结构的数据源（PDF/Word/在线文档等等不同形式），解析难度大。针对不同结构的文档需要采用不同的解析方法或工具。

**方法1：提升原始业务知识文档质量**

业务如果在做文档解析技术实现前，提供良好格式与形式的原始文档，可以降低文档解析出错的概率，提高大模型最终的检索准确率。

文档形式上有如下建议：
- **使用语法简单的格式**：以markdown/html等语法格式简单的文档形式为佳。PDF/PPT等文档格式存在较大的复杂性，非必要不使用。
- **保证文档风格统一**：确保文件内的分段标识明确，比如使用标题、副标题和正文等不同层级的标识，有助于后续的处理和解析。


单个文档排版结构上，应当简化：
- **简化布局**：确保各区域（如文本块、图像、表格和图表等）清晰可辨，不交叉。减少双栏、三栏等多栏目布局的使用。
- **简化表格**：表格尽量不要复杂化，减少跨页的长表格使用，减少单元格上下合并/左右合并/多单元格合并等形式，并且单元格的行、列采用统一的对齐方式。
- **用文本替代图片**：如果文本能够替代图片，建议使用文本形式，减少图片内容无法解析带来的问题。
- **不要使用内嵌文档**：对于Word等支持内嵌文档形式的文档，应避免使用内嵌文档。


**方法2：选择合适的解析方法与工具**

Markdown格式因其简洁性和易于解析的特点，被广泛认为是LLM（大型语言模型）友好的文档格式。Markdown通过明确的标记语法，帮助模型更好地理解文档结构和内容，从而提高信息提取的准确性和效率。

**因此，建议将不同类型结构的数据，通过合适的解析工具处理成markdown格式。**


在本文的案例中，你已经学会使用了阿里云百炼提供的智能文件解析器工具和大模型，帮助你提升你的开发效率。这些工具具有较高的普适性，可以帮助你少写业务代码并快速解决大部分业务问题。

*   [DashScopeParse](https://help.aliyun.com/zh/model-studio/developer-reference/dashscopeparse)：基于 LlamaIndex框架的文件解析器，能够将常见的 doc/docx/pdf 文件解析成大模型易于处理的 markdown 文本，并且能够解析文档里的图片信息。你可以将它无缝集成至课程 LlamaIndex项目中，最关键的是，这个接口是免费的。
    
*   [通义千问API](https://help.aliyun.com/zh/model-studio/developer-reference/use-qwen-by-calling-api)：使用文本生成、视觉理解模型辅助进行内容优化。
    
你也可以根据您的业务需求现状，开发更匹配你的业务文档的解析工具，达到效率与成本的平衡。

一些常见的解析器示例（如下仅供参考，您也可以自行在互联网查找类似工具）：

- PDF文档：

In [ ]:
! pip install PyMuPDF Pillow

In [47]:
import fitz  # PyMuPDF
import os

# 定义一个函数来判断文本是否是标题
def is_title(text, font_size):
    # 假设标题的字体通常比正文大, 这里可以根据具体文档调整
    return font_size > 12

def convert_pdf_to_markdown(pdf_path):
    # 打开PDF文件
    pdf_document = fitz.open(pdf_path)
    markdown_content = ""

    # 提取每一页的文本
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for block in blocks:
            for line in block["lines"]:
                for span in line["spans"]:
                    text = span["text"].strip()
                    font_size = span["size"]
                    if is_title(text, font_size):
                        # 添加一级标题
                        markdown_content += f"# {text}\n\n"
                    else:
                        # 将每一段内容添加到Markdown中
                        markdown_content += text + "\n\n"
    return markdown_content

# 示例用法
pdf_path = "./docs/测试文档1.pdf"
markdown_content = convert_pdf_to_markdown(pdf_path)
print(markdown_content)

# 内容开发⼯程师

# 岗位类型

⼤类：技术⼤类

细分类型：综合技术岗位

# ⼯作职责

# 核⼼职责

结合教育理论与技术实践，通过⾼质量的内容创造⽀持学习者的成⻓与发展。

# 详细职责

# 1. 内容研究与分析

对最新的教育技术趋势、学习理论和市场需求进⾏深⼊研究。这包括分析竞争对⼿的产品，

评估现有教育资源的有效性，并探索如何将新兴技术（如⼈⼯智能、虚拟现实等）整合进我

们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并

能够满⾜教育者和学习者的真实需求。

# 2. 教材和课程开发

根据研究和市场反馈，我将设计和开发⾼质量的教育教材和课程。这包括撰写教学⼤纲、制

作课件、设计评估⼯具等。我的职责还包括确保内容符合教育标准和学习⽬标，以提供全⾯

的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习⻛格和⽔平。

# 3. 内容优化与更新

在内容开发过程中，我会不断优化已有的教育材料。通过跟踪学习者的反馈和评价，我能够

识别出内容中的潜在问题，并及时进⾏调整。此外，我也会定期更新材料，以反映新的研究

成果、技术进步和市场变化。保持内容的时效性和相关性是我的重要任务之⼀。




由于PDF的格式特点，很难准确判别出目录的层级结构逻辑。你可以参照本文案例，将结果输入到大模型，让大模型帮助你解决这一复杂的业务开发过程，简化业务代码并提升开发效率。

- Word文档：

In [ ]:
! pip install mammoth

In [42]:
import mammoth

# 打开Word文档
with open("./docs/测试文档1.docx", "rb") as docx_file:
    # 转换为Markdown
    result = mammoth.convert_to_markdown(docx_file)
    markdown_content = result.value  # 获取Markdown内容

print(markdown_content)

# 内容开发工程师

## 岗位类型

大类：技术大类

细分类型：综合技术岗位

## 工作职责

### 核心职责

结合教育理论与技术实践，通过高质量的内容创造支持学习者的成长与发展。

### 详细职责

#### 1\. 内容研究与分析

对最新的教育技术趋势、学习理论和市场需求进行深入研究。这包括分析竞争对手的产品，评估现有教育资源的有效性，并探索如何将新兴技术（如人工智能、虚拟现实等）整合进我们的教育内容中。通过持续的市场调研，我能够确保我们的内容在技术上始终处于前沿，并能够满足教育者和学习者的真实需求。

#### 2\. 教材和课程开发

根据研究和市场反馈，我将设计和开发高质量的教育教材和课程。这包括撰写教学大纲、制作课件、设计评估工具等。我的职责还包括确保内容符合教育标准和学习目标，以提供全面的学习体验。同时，我会考虑不同学习者的需求，确保内容能够适应各种学习风格和水平。

#### 3\. 内容优化与更新

在内容开发过程中，我会不断优化已有的教育材料。通过跟踪学习者的反馈和评价，我能够识别出内容中的潜在问题，并及时进行调整。此外，我也会定期更新材料，以反映新的研究成果、技术进步和市场变化。保持内容的时效性和相关性是我的重要任务之一。




- 在线文档：

In [ ]:
! pip install html2text

In [43]:
import requests
import html2text

# 获取URL内容
def get_html_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

# 将HTML转换为Markdown
def convert_html_to_markdown(html_content):
    h = html2text.HTML2Text()
    h.ignore_links = False  # 保留链接
    markdown_content = h.handle(html_content)
    return markdown_content

# 主函数
def url_to_markdown(url):
    html_content = get_html_from_url(url)
    markdown_content = convert_html_to_markdown(html_content)
    return(markdown_content)

# 示例用法
url = "https://help.aliyun.com/zh/model-studio/developer-reference/dashscopeparse"
print(url_to_markdown(url))

[](https://www.aliyun.com/)

__

[产品](https://www.aliyun.com/product/list)
解决方案[文档与社区](https://help.aliyun.com/)[权益中心](https://www.aliyun.com/benefit)[定价](https://www.aliyun.com/price)[云市场](https://market.aliyun.com/)[合作伙伴](https://partner.aliyun.com/management/v2)[支持与服务](https://www.aliyun.com/service)[了解阿里云](https://www.aliyun.com/about)

 __

__

[__](https://help.aliyun.com/search)

____[AI 助理](https://www.aliyun.com/ai-
assistant)[备案](https://beian.aliyun.com/)[控制台](https://home.console.aliyun.com/)

文档

[产品文档](/)

输入文档关键字查找 __

[大模型服务平台百炼](/zh/model-studio/)

__

__

  * [ __ 开始使用 ](/zh/model-studio/getting-started/)
  * [ __ 功能特性 ](/zh/model-studio/user-guide/)
  * [ __ 实践教程 ](/zh/model-studio/use-cases/)
  * [ __ 开发参考 ](/zh/model-studio/developer-reference/)
  * [ __ 计费说明 ](/zh/model-studio/billing/)
  * [ __ 服务支持 ](/zh/model-studio/support/)

__

[首页](/) __ [大模型服务平台百炼](/zh/model-studio/) __ [开发参考](/zh/model-
studio/developer-reference/) __ [LlamaIndex](/zh/model-studio/develop

可以看到上面的在线文档解析后的内容包含许多冗余的信息。通常来讲，对于在线文档，我们需要进行对这种冗余数据进行进一步处理，再进行后续处理。在你学会使用大模型之后，你可以试着将结果输入给大模型，让大模型帮助你完成处理或者代码生成。

<style>
    table {
      width: 100%;
      margin: 20px auto; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
### 5.2. 文档切片阶段

在大模型RAG过程中，适当的文档切片方法可以提高系统的效果和效率。根据具体的文档类型和需求，选择合适的切片方式非常重要。

常见的文档切片方法、实现方式、优点、缺点、适用场景总结如下。

| 切片方法 | 实现方式                | 优点                                | 缺点           | 适用场景              |
|-----|----------------|-------------------|--------------|-------------|
| 基于字符切片       | 将文本按照固定字符如,进行分割                                                  | 精细度高，适用于非常细粒度的分析和处理                                                            | 信息量较少，可能导致上下文丢失                                                                    | 需要极高精度的文本处理场景                       |
| 基于token长度切片 | 使用tokenizer将文本切分成tokens，按照指定的最大token长度进行分割并保留原始位置的对应关系           | 保留原始输入和切片后的位置对应关系，确保上下文一致，适用于长文本                                  | 依赖tokenizer的准确性和性能，可能需要额外的处理步骤                                               | 处理超长文本，并需要保持token的精确位置关系的场景|
| 基于句子切片       | 利用标点符号（如句号、问号）将文本分割为单个句子                                                   | 能够保持完整的语义单元，易于理解和处理                                                            | 可能会丢失段落级别的上下文信息                                                                   | 一般文本处理和分析            |
| 基于段落切片       | 根据换行符或段落标记将文本按段落分割                                                               | 保持较大的上下文信息，利于理解整段内容                                                            | 粒度较大，可能包含过多无关信息                                                                    | 需要理解复杂信息的场景，如文档摘要和问答         |
| 基于文档结构切片   | 解析文档的结构（如标题、子标题、列表）并基于这些结构进行分割                                         | 能够识别更多更上层的版面元素，有利于后续切片                                                      | 速度慢，需要识别全文，识别过程黑盒，适用场景有限                                                   | 需要识别和处理复杂文档结构的场景                 |
| 基于语义切片       | 使用自然语言处理(NLP)技术，如主题模型或BERT，来识别语义边界并进行分割                               | 能够通过系统地对数据进行切片和切块，解决复杂表单元问题，增强表格问答的准确性                      | 实现复杂，可能需要更多计算资源                                                                    | 涉及多条信息的复杂表单处理，增强表格问答         |
| 从小到大分块       | 结合多种切片方式，将文档从小到大不同尺寸的分块存储在向量数据库中，进行多尺度信息检索 | 综合了小分块和大分块的优点，能够提供多层次的细粒度信息和上下文                                    | 实现复杂，存储和计算需求较高                                                                      | 需要在不同层次上进行精细和上下文深度结合的场景   |


本文案例中的图片处理、基于语义切片文本、表格heading切片等问题，是非常复杂的问题，应用到了多种切片方式，包括基于字符、基于句子、基于段落、基于文档结构、基于语义等多种切片方式。但是这些我们都是通过大模型来快速实现的，可以看到，大模型为复杂业务开发提供了新的解决问题的思路。

对于案例中的部分处理节点，如果你的业务语料/格式比较固定，你也可以通过业务代码的方式解决问题。llamaindex等框架提供了现成的[Splitters工具](https://docs.llamaindex.ai/en/stable/api_reference/node_parsers/)，你可直接使用。


<style>
    table {
      width: 100%;
      margin: 20px auto; /* Center the table */
      border-collapse: collapse; /* Collapse borders for a cleaner look */
      font-family: sans-serif; 
    }

    th, td {
      padding: 10px;
      text-align: left;
      border: 1px solid #ddd; /* Light gray border */
    }

    th {
      background-color: #f2f2f2; /* Light gray background for header */
      font-weight: bold;
    }

    tr:nth-child(even) { /* Zebra striping */
      background-color: #f9f9f9;
    }

    tr:hover { /* Highlight row on hover */
      background-color: #e0f2ff; /* Light blue */
    }
</style>
### 5.3. embedding嵌入与向量存储检索阶段

| 过程               | 常见问题                                                                                          | 使用建议                                                                                           |
|--------------------|----------------------------------------------|--------------------------------------------------------|
| embedding嵌入     | 1. Embedding质量差，未能准确捕捉语义。 <br>2. 高维向量计算复杂，耗费资源。 <br>   3. 数据量大时生成embedding耗时长。    | 1. 使用更先进的embedding模型（如[modelscope上的embedding模型](https://modelscope.cn/models?name=embedding)），并根据具体任务需求选择是否对模型进行微调。  <br>2. 采用降维技术（如PCA、t-SNE）来降低计算复杂度。<br> 3. 利用分布式计算框架（如Spark）并行处理大规模数据。          |
| 向量数据存储       | 1. 存储空间需求大。   <br>2. 数据库检索速度慢，影响性能。<br>  3. 数据一致性和备份问题。 | 1. 使用高效的压缩算法（如量化技术）来减少存储空间。 <br>2. 采用高性能向量数据库（如Faiss、Annoy）及优化索引结构（如HNSW）。 <br>3. 定期备份数据，使用分布式存储系统（如HDFS）提高数据一致性和可靠性。        |
| 向量检索           | 1. 检索精度不足，返回结果不准确。<br>2. 检索速度慢，处理大规模数据时效率低。<br>  3. 检索结果排序不合理，用户体验差。   | 1. 选择适当的检索算法（如LSH、HNSW）提高检索精度，调整相似度阈值。<br>2. 利用分布式检索系统（如Elasticsearch、Milvus）提升检索速度和扩展性。<br> 3. 结合上下文信息和用户反馈对检索结果进行排序优化，使用学习排序方法（Learning to Rank）。                              |



## ✅ 本节小结

本节我们优化了我们的新人答疑机器人，提升知识库问答方面的表现，让其能够应对图片内容问答、表格内容问答、复杂目录结构的文档内容问答等等问题。为了持续改进我们的 RAG 应用，我们还应当构建一套严谨的评测指标体系，在后续章节中，我们将介绍RAGAS评测来应对自动化评测的问题。

这里我们先简要介绍一下RAG评测的主要做法，具体内容可以学习后面的章节：

- 为了评估RAG系统（检索增强生成系统）的性能，我们会邀请业务专家参与评测，共同设计涵盖多种业务场景的测试用例。评测重点考察系统响应速度、回答准确性以及对用户意图的理解程度。通过全面科学的评估，我们可以找出系统的优势和不足，从而帮助开发者改进系统，使其更好地满足业务需求。

- RAG系统通常包含检索和生成两个模块。评测时，可以分别评估这两个模块，建立针对性的评价标准和实施方法，也可以采用端到端的方式，直接评估最终输出效果。在指标设计方面，针对检索模块，我们主要关注准确率、召回率和F1值等指标，衡量其检索的准确性；针对生成模块，我们主要关注相关性和真实性等指标，衡量生成答案的价值。

- 此外，评测可以参考业界通用的评估策略，例如Ragas提供的评测矩阵指南，也可以根据实际需求自定义指标。这些方法将有助于量化和改进每个模块的性能。

## 🔥 课后小测验

【单选题】2.3.1. 在对长文本进行切片时，以下哪种策略最能保留文本的语义完整性，并避免信息丢失？（ ）

A. 按固定字符数切分，例如每500个字符切成一个chunk。

B. 按句子边界切分，确保每个chunk包含完整的句子。

C. 按段落边界切分，并根据段落长度进行适当调整，避免过长或过短的chunk。

D. 随机切分文本，确保每个chunk的长度大致相同。

答案：C

解析： 段落通常代表一个相对完整的语义单元。按段落切分可以最大程度地保留文本的语义信息，同时根据段落长度进行调整可以避免chunk过长或过短，从而更好地适应大模型的输入限制。

<br>

【多选题】2.3.2. 优化文档切片方式的整体思路是什么？（ ）

A. 尽可能多的保留原始文档信息

B. 基于 markdown 文档结构进行语义切片

C. 为独立的内容补充语义信息，使信息更独立和完整

D. 尽可能的缩短文本长度

E. 将所有内容合并成一个大的 chunk

F. 随机切分文档

答案：B, C

解析： 优化的思路是按照 markdown 文档结构进行语义切片，并为独立的内容（如标题、段落、单元格内容）补充语义信息，使其更独立和完整，以便大模型更好地理解。